In [19]:
import torch
import torch.nn as nn

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [21]:
abc = torch.rand(5, 6, 2)
abc

tensor([[[0.6502, 0.3705],
         [0.6517, 0.7299],
         [0.5901, 0.7478],
         [0.9050, 0.5970],
         [0.7686, 0.0793],
         [0.0625, 0.6381]],

        [[0.0045, 0.3661],
         [0.5866, 0.2329],
         [0.0878, 0.5366],
         [0.0215, 0.4228],
         [0.5738, 0.6753],
         [0.2018, 0.4452]],

        [[0.6987, 0.7413],
         [0.5910, 0.9420],
         [0.5545, 0.3177],
         [0.1659, 0.9551],
         [0.3505, 0.1483],
         [0.2049, 0.5989]],

        [[0.8580, 0.6703],
         [0.4122, 0.4105],
         [0.1906, 0.5165],
         [0.3777, 0.3244],
         [0.1858, 0.0507],
         [0.6930, 0.0928]],

        [[0.8017, 0.5512],
         [0.4507, 0.2722],
         [0.1689, 0.7476],
         [0.8358, 0.7434],
         [0.6004, 0.5159],
         [0.9699, 0.2660]]])

In [22]:
input = abc.permute(0,2,1)
input

tensor([[[0.6502, 0.6517, 0.5901, 0.9050, 0.7686, 0.0625],
         [0.3705, 0.7299, 0.7478, 0.5970, 0.0793, 0.6381]],

        [[0.0045, 0.5866, 0.0878, 0.0215, 0.5738, 0.2018],
         [0.3661, 0.2329, 0.5366, 0.4228, 0.6753, 0.4452]],

        [[0.6987, 0.5910, 0.5545, 0.1659, 0.3505, 0.2049],
         [0.7413, 0.9420, 0.3177, 0.9551, 0.1483, 0.5989]],

        [[0.8580, 0.4122, 0.1906, 0.3777, 0.1858, 0.6930],
         [0.6703, 0.4105, 0.5165, 0.3244, 0.0507, 0.0928]],

        [[0.8017, 0.4507, 0.1689, 0.8358, 0.6004, 0.9699],
         [0.5512, 0.2722, 0.7476, 0.7434, 0.5159, 0.2660]]])

In [23]:
batch, channel, seq_len = input.shape
segments = seq_len // 3
input1 = input.view(batch, channel, segments, 3)
input1

tensor([[[[0.6502, 0.6517, 0.5901],
          [0.9050, 0.7686, 0.0625]],

         [[0.3705, 0.7299, 0.7478],
          [0.5970, 0.0793, 0.6381]]],


        [[[0.0045, 0.5866, 0.0878],
          [0.0215, 0.5738, 0.2018]],

         [[0.3661, 0.2329, 0.5366],
          [0.4228, 0.6753, 0.4452]]],


        [[[0.6987, 0.5910, 0.5545],
          [0.1659, 0.3505, 0.2049]],

         [[0.7413, 0.9420, 0.3177],
          [0.9551, 0.1483, 0.5989]]],


        [[[0.8580, 0.4122, 0.1906],
          [0.3777, 0.1858, 0.6930]],

         [[0.6703, 0.4105, 0.5165],
          [0.3244, 0.0507, 0.0928]]],


        [[[0.8017, 0.4507, 0.1689],
          [0.8358, 0.6004, 0.9699]],

         [[0.5512, 0.2722, 0.7476],
          [0.7434, 0.5159, 0.2660]]]])

In [24]:
channel_input1 = input1[:,1,:,:]
channel_input1

tensor([[[0.3705, 0.7299, 0.7478],
         [0.5970, 0.0793, 0.6381]],

        [[0.3661, 0.2329, 0.5366],
         [0.4228, 0.6753, 0.4452]],

        [[0.7413, 0.9420, 0.3177],
         [0.9551, 0.1483, 0.5989]],

        [[0.6703, 0.4105, 0.5165],
         [0.3244, 0.0507, 0.0928]],

        [[0.5512, 0.2722, 0.7476],
         [0.7434, 0.5159, 0.2660]]])

In [44]:
linear = nn.Linear(in_features=3, out_features=3, bias=False)
linear.weight = nn.Parameter(nn.init.orthogonal_(linear.weight))
out = linear(channel_input1)
out.shape

TypeError: Linear.__init__() got an unexpected keyword argument 'requires_grad_'

In [26]:
w33 = torch.rand(10,1, 5,21)
print(w33.shape)
w33.squeeze(1).shape


torch.Size([10, 1, 5, 21])


torch.Size([10, 5, 21])

In [40]:
w34 = torch.rand(5, 6)
w34

tensor([[0.6887, 0.0848, 0.9919, 0.6378, 0.7851, 0.3636],
        [0.4303, 0.0417, 0.3417, 0.7909, 0.7361, 0.9721],
        [0.4405, 0.7878, 0.5268, 0.0902, 0.6004, 0.7472],
        [0.8127, 0.8428, 0.4446, 0.0099, 0.4628, 0.9326],
        [0.1924, 0.1868, 0.9922, 0.1819, 0.6936, 0.1372]])

In [41]:
w34 = torch.unsqueeze(w34,1)
print(w34.shape)
w34

torch.Size([5, 1, 6])


tensor([[[0.6887, 0.0848, 0.9919, 0.6378, 0.7851, 0.3636]],

        [[0.4303, 0.0417, 0.3417, 0.7909, 0.7361, 0.9721]],

        [[0.4405, 0.7878, 0.5268, 0.0902, 0.6004, 0.7472]],

        [[0.8127, 0.8428, 0.4446, 0.0099, 0.4628, 0.9326]],

        [[0.1924, 0.1868, 0.9922, 0.1819, 0.6936, 0.1372]]])

In [42]:
w34 = w34.repeat(1,2,1)
w34

tensor([[[0.6887, 0.0848, 0.9919, 0.6378, 0.7851, 0.3636],
         [0.6887, 0.0848, 0.9919, 0.6378, 0.7851, 0.3636]],

        [[0.4303, 0.0417, 0.3417, 0.7909, 0.7361, 0.9721],
         [0.4303, 0.0417, 0.3417, 0.7909, 0.7361, 0.9721]],

        [[0.4405, 0.7878, 0.5268, 0.0902, 0.6004, 0.7472],
         [0.4405, 0.7878, 0.5268, 0.0902, 0.6004, 0.7472]],

        [[0.8127, 0.8428, 0.4446, 0.0099, 0.4628, 0.9326],
         [0.8127, 0.8428, 0.4446, 0.0099, 0.4628, 0.9326]],

        [[0.1924, 0.1868, 0.9922, 0.1819, 0.6936, 0.1372],
         [0.1924, 0.1868, 0.9922, 0.1819, 0.6936, 0.1372]]])

In [43]:
w34.shape

torch.Size([5, 2, 6])

In [45]:
w34.view(5, -1)

tensor([[0.6887, 0.0848, 0.9919, 0.6378, 0.7851, 0.3636, 0.6887, 0.0848, 0.9919,
         0.6378, 0.7851, 0.3636],
        [0.4303, 0.0417, 0.3417, 0.7909, 0.7361, 0.9721, 0.4303, 0.0417, 0.3417,
         0.7909, 0.7361, 0.9721],
        [0.4405, 0.7878, 0.5268, 0.0902, 0.6004, 0.7472, 0.4405, 0.7878, 0.5268,
         0.0902, 0.6004, 0.7472],
        [0.8127, 0.8428, 0.4446, 0.0099, 0.4628, 0.9326, 0.8127, 0.8428, 0.4446,
         0.0099, 0.4628, 0.9326],
        [0.1924, 0.1868, 0.9922, 0.1819, 0.6936, 0.1372, 0.1924, 0.1868, 0.9922,
         0.1819, 0.6936, 0.1372]])